In [6]:
import numpy as np
import pandas as pd

from utils import get_train_data
from FeatureEngineering import _encode
# We will do categorical encoding, and not use the integrated module of XGBoost
# supposed to handle these categorical variables

from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

### __Setup__

In [13]:
# We import the data:
X, y = get_train_data()

# We put the label to the log to help the model:
X_encoded, y_log = _encode(X, y)

# Pipeline creation:
model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=6,
        random_state=42
    )

### __Model prediction__

In [14]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_log, 
                                                    test_size=0.4, 
                                                    random_state=42)

# Model fit
model.fit(X_train, y_train)

# Model predict
y_pred = model.predict(X_test)

# Error estimation
error = root_mean_squared_error(y_test, y_pred)
print(f'The RMSE is {error}.')

The RMSE is 0.13453803415959517.


### __Test Data Prediction__

In [16]:
test_data = pd.read_csv('data/test.csv')

# Preprocess the test data using the same pipeline
test_data_encoded = _encode(test_data)
test_prediction = model.predict(test_data_encoded)

# Make predictions on the test data
predictions = np.exp(test_prediction)

TypeError: _encode() missing 1 required positional argument: 'y'

### __Output extraction__

In [17]:
results = pd.DataFrame(
    dict(
        Id=test_data['Id'],
        SalePrice=predictions,
    )
)
results.to_csv("submission.csv", index=False)